# TSA Chapter 1: White Noise Processes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch1/TSA_ch1_white_noise/TSA_ch1_white_noise.ipynb)

This notebook demonstrates:
- White noise definition and properties
- ACF of white noise (all within confidence bands)
- Three types: weak, strong (iid), Gaussian
- Ljung-Box test for white noise

In [ ]:
!pip install yfinance matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.stattools import acf
from statsmodels.stats.diagnostic import acorr_ljungbox

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def generate_white_noise(n, sigma=1.0):
    return np.random.normal(0, sigma, n)

In [ ]:
# Chart: ch1_def_white_noise
# White noise process with ACF
np.random.seed(42)
n = 300
wn = generate_white_noise(n)

fig, axes = plt.subplots(1, 2, figsize=(8, 2.8))

axes[0].plot(wn, color=COLORS['blue'], linewidth=0.5, alpha=0.8)
axes[0].axhline(0, color=COLORS['red'], linewidth=0.8, linestyle='--', label=r'$\mu = 0$')
axes[0].axhline(2, color=COLORS['orange'], linewidth=0.6, linestyle=':', label=r'$\pm 2\sigma$')
axes[0].axhline(-2, color=COLORS['orange'], linewidth=0.6, linestyle=':')
axes[0].set_title(r'White noise: $\varepsilon_t \sim WN(0, \sigma^2)$', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel(r'$\varepsilon_t$')
axes[0].legend(fontsize=7, loc='upper right', frameon=False)

acf_vals = acf(wn, nlags=25)
axes[1].bar(range(len(acf_vals)), acf_vals, color=COLORS['blue'], width=0.5, alpha=0.7)
ci = 1.96 / np.sqrt(n)
axes[1].axhline(ci, color=COLORS['red'], linewidth=0.8, linestyle='--', label=r'$\pm 1.96/\sqrt{T}$')
axes[1].axhline(-ci, color=COLORS['red'], linewidth=0.8, linestyle='--')
axes[1].axhline(0, color=COLORS['gray'], linewidth=0.3)
axes[1].set_title('ACF white noise', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel(r'$\hat{\rho}(h)$')
axes[1].legend(fontsize=7, loc='upper right', frameon=False)

fig.tight_layout(w_pad=2.0)
save_chart(fig, 'ch1_def_white_noise')
plt.show()

In [ ]:
# Chart: ch1_white_noise_test
# White noise: series, ACF, and histogram
np.random.seed(42)
n = 300
wn = generate_white_noise(n)

fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))

axes[0].plot(wn, color=COLORS['blue'], linewidth=0.4)
axes[0].axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
axes[0].set_title('White noise', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel(r'$\varepsilon_t$')

acf_vals = acf(wn, nlags=20)
axes[1].bar(range(len(acf_vals)), acf_vals, color=COLORS['blue'], width=0.5, alpha=0.7)
ci = 1.96 / np.sqrt(n)
axes[1].axhline(ci, color=COLORS['red'], linewidth=0.8, linestyle='--', label='95% CI')
axes[1].axhline(-ci, color=COLORS['red'], linewidth=0.8, linestyle='--')
axes[1].set_title('ACF (all within bands)', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('ACF')
axes[1].legend(fontsize=7, loc='upper right', frameon=False)

axes[2].hist(wn, bins=30, color=COLORS['blue'], alpha=0.6, edgecolor='white', density=True)
x_r = np.linspace(-4, 4, 100)
axes[2].plot(x_r, stats.norm.pdf(x_r, 0, 1), color=COLORS['red'], linewidth=1.5, label='N(0,1)')
axes[2].set_title('Distribution', fontsize=9, fontweight='bold')
axes[2].set_xlabel(r'$\varepsilon_t$')
axes[2].set_ylabel('Density')
axes[2].legend(fontsize=7, loc='upper right', frameon=False)

fig.tight_layout(w_pad=2.0)
save_chart(fig, 'ch1_white_noise_test')
plt.show()

In [ ]:
# Chart: ch1_white_noise_types
# 3 types of white noise: weak, strong (iid), Gaussian
np.random.seed(42)
n = 300

# 1) Weak WN: uncorrelated but dependent (GARCH-like)
z = np.random.normal(0, 1, n)
sigma2 = np.ones(n)
for t in range(1, n):
    sigma2[t] = 0.1 + 0.7 * z[t-1]**2 * sigma2[t-1]
weak_wn = np.sqrt(sigma2) * np.random.normal(0, 1, n)
weak_wn = (weak_wn - weak_wn.mean()) / weak_wn.std()

# 2) Strong WN (iid): t-distribution (non-Gaussian but iid)
strong_wn = stats.t.rvs(df=5, size=n, random_state=123)
strong_wn = (strong_wn - strong_wn.mean()) / strong_wn.std()

# 3) Gaussian WN: N(0,1)
gauss_wn = np.random.normal(0, 1, n)

fig, axes = plt.subplots(1, 3, figsize=(9, 2.5))

titles = ['Weak white noise', 'Strong white noise (i.i.d.)', 'Gaussian white noise']
data = [weak_wn, strong_wn, gauss_wn]
colors = [COLORS['orange'], COLORS['blue'], COLORS['green']]
subtitles = ['Uncorrelated, nonlinear dependence', r'$\varepsilon_t \sim t_5$ (i.i.d.)',
             r'$\varepsilon_t \sim N(0,1)$ (i.i.d.)']

for i, (ax, d, c, t_title, sub) in enumerate(zip(axes, data, colors, titles, subtitles)):
    ax.plot(d, color=c, linewidth=0.4, alpha=0.8)
    ax.axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
    ax.set_title(t_title, fontsize=8, fontweight='bold')
    ax.set_xlabel('Time', fontsize=7)
    ax.set_ylabel(r'$\varepsilon_t$', fontsize=7)
    ax.tick_params(labelsize=6)
    ax.text(0.5, -0.22, sub, transform=ax.transAxes, fontsize=7,
            ha='center', va='top', style='italic', color=COLORS['gray'])

fig.tight_layout(w_pad=2.5)
fig.subplots_adjust(bottom=0.22)
save_chart(fig, 'ch1_white_noise_types')
plt.show()

In [ ]:
# Ljung-Box Test Results
np.random.seed(42)
n = 500
wn = generate_white_noise(n)

# AR(1) for comparison
ar1 = np.zeros(n)
phi = 0.8
for t in range(1, n):
    ar1[t] = phi * ar1[t-1] + np.random.normal(0, 1)

lb_wn = acorr_ljungbox(wn, lags=[10, 20], return_df=True)
lb_ar1 = acorr_ljungbox(ar1, lags=[10, 20], return_df=True)

print('='*60)
print('LJUNG-BOX TEST RESULTS')
print('='*60)
print(f'\nH0: Data is white noise (no autocorrelation)')
print(f'\nWhite noise series:')
print(f'  Lag 10: Q = {lb_wn.loc[10, "lb_stat"]:.2f}, p-value = {lb_wn.loc[10, "lb_pvalue"]:.4f}')
print(f'  Lag 20: Q = {lb_wn.loc[20, "lb_stat"]:.2f}, p-value = {lb_wn.loc[20, "lb_pvalue"]:.4f}')
print(f'  -> p > 0.05: FAIL TO REJECT H0 (consistent with white noise)')
print(f'\nAR(1) series:')
print(f'  Lag 10: Q = {lb_ar1.loc[10, "lb_stat"]:.2f}, p-value = {lb_ar1.loc[10, "lb_pvalue"]:.6f}')
print(f'  Lag 20: Q = {lb_ar1.loc[20, "lb_stat"]:.2f}, p-value = {lb_ar1.loc[20, "lb_pvalue"]:.6f}')
print(f'  -> p < 0.05: REJECT H0 (NOT white noise)')